In [1]:
import sys
sys.path.append('/Users/aimeebarciauskas/Library/Python/3.6/lib/python/site-packages')
sys.path.append('/Users/aimeebarciauskas/Library/Python/3.7/lib/python/site-packages')
sys.path.append('/Users/aimeebarciauskas/Projects/ifsc/')
import psycopg2
import pycountry
import helpers

/Users/aimeebarciauskas/Library/Python/3.6/lib/python/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
query = """
select count(*), athletes.nation as nation, comps.year as year from bouldering_comp_results
  join athletes on athletes.id = bouldering_comp_results.athlete_id
  join comps on bouldering_comp_results.comp_id = comps.id
where bouldering_comp_results.comp_stage = 'semifinal'
group by nation, year
order by nation, year;
"""

countries_semifinals_counts = helpers.raw_fetch(query)
countries_semifinals_counts[0:10]

[(6, 'AUS', 2007),
 (1, 'AUS', 2009),
 (2, 'AUS', 2010),
 (3, 'AUS', 2011),
 (3, 'AUS', 2012),
 (4, 'AUS', 2013),
 (9, 'AUS', 2014),
 (3, 'AUS', 2015),
 (1, 'AUS', 2016),
 (27, 'AUT', 2007)]

In [25]:
query = """
select distinct(athletes.nation) from bouldering_comp_results
  join athletes on athletes.id = bouldering_comp_results.athlete_id
where bouldering_comp_results.comp_stage = 'semifinal';
"""

nations = helpers.format_rows(helpers.raw_fetch(query))
nations

['NED',
 'VEN',
 'USA',
 'SRB',
 'HUN',
 'SUI',
 'AUT',
 'IRL',
 'RUS',
 'ISR',
 'GBR',
 'KAZ',
 'FIN',
 'HKG',
 'BEL',
 'SLO',
 'CZE',
 'GER',
 'LTU',
 'LAT',
 'UKR',
 'POL',
 'SVK',
 'SWE',
 'ESP',
 'BLR',
 'MAS',
 'AUS',
 'FRA',
 'BUL',
 'CHN',
 'CAN',
 'TPE',
 'IRI',
 'KOR',
 'ITA',
 'NOR',
 'JPN',
 'MEX']

In [30]:
# fill in missing years with 0
year_range = range(2007, 2018)
# Verified with https://en.wikipedia.org/wiki/List_of_IOC_country_codes
alternate_keys = {
    'GER': 'Germany', #DEU
    'BUL': 'Bulgaria', #BGR
    'IRI': 'Iran', #IRN
    'LAT': 'Latvia', #LVA
    'MAS': 'Malaysia', #MYS
    'NED': 'Netherlands', #NLD
    'SLO': 'Slovenia', #SVN
    'SUI': 'Switzerland', #CHE?
    'TPE': 'Taipei' # Not listed
#     'DEN': 'Denmark', #DNK
#     'LIB': 'Libya', #LBY, src https://en.wikipedia.org/wiki/Country_codes:_L
#     'POR': 'Portugal', #PRT
#     'NEP': ''
}

def country_year_map(triplet):
    alpha_3 = triplet[1]
    country = alpha_3
    try:
        country = pycountry.countries.get(alpha_3=alpha_3).name
    except KeyError as e:
        #print('Country for 3-letter not found: {0}'.format(alpha_3))
        country = alternate_keys[alpha_3]

    return {
        'count': triplet[0],
        'alpha_3': alpha_3,
        'country': country,
        'year': triplet[2]
    }

def format_key(alpha_3, year):
    return '{0}_{1}'.format(alpha_3, year)

country_count_maps = list(map(country_year_map, countries_semifinals_counts))
country_count_map_of_maps = {format_key(i['alpha_3'], i['year']): i for i in country_count_maps}

for year in year_range:
    for alpha_3 in nations:
        cy_key = format_key(alpha_3, year)
        if cy_key not in country_count_map_of_maps:
            country_count_map_of_maps[cy_key] = country_year_map([0, alpha_3, year])
            
country_count_map_values = country_count_map_of_maps.values()

In [46]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
import itertools

country_count_map_of_maps

{'AUS_2007': {'alpha_3': 'AUS',
  'count': 6,
  'country': 'Australia',
  'year': 2007},
 'AUS_2008': {'alpha_3': 'AUS',
  'count': 0,
  'country': 'Australia',
  'year': 2008},
 'AUS_2009': {'alpha_3': 'AUS',
  'count': 1,
  'country': 'Australia',
  'year': 2009},
 'AUS_2010': {'alpha_3': 'AUS',
  'count': 2,
  'country': 'Australia',
  'year': 2010},
 'AUS_2011': {'alpha_3': 'AUS',
  'count': 3,
  'country': 'Australia',
  'year': 2011},
 'AUS_2012': {'alpha_3': 'AUS',
  'count': 3,
  'country': 'Australia',
  'year': 2012},
 'AUS_2013': {'alpha_3': 'AUS',
  'count': 4,
  'country': 'Australia',
  'year': 2013},
 'AUS_2014': {'alpha_3': 'AUS',
  'count': 9,
  'country': 'Australia',
  'year': 2014},
 'AUS_2015': {'alpha_3': 'AUS',
  'count': 3,
  'country': 'Australia',
  'year': 2015},
 'AUS_2016': {'alpha_3': 'AUS',
  'count': 1,
  'country': 'Australia',
  'year': 2016},
 'AUS_2017': {'alpha_3': 'AUS',
  'count': 0,
  'country': 'Australia',
  'year': 2017},
 'AUT_2007': {'alpha_